In [ ]:
# ------------------------------------------------------------
# PROYECTO FINAL – MODELO DE CLASIFICACIÓN DE DIBUJOS
# JAVIEL ALEXANDER HIDALGO – MATRÍCULA 21-EISN-2-019
#
# En este proyecto estoy desarrollando un modelo capaz de reconocer
# dibujos hechos a mano combinando imágenes del dataset MNIST y 
# dibujos del conjunto QuickDraw. La idea es entrenar una red neuronal
# sencilla que pueda distinguir entre los estilos de ambos tipos de 
# imágenes, y más adelante integrarla con una aplicación que permita 
# subir un dibujo y clasificarlo.
# ------------------------------------------------------------


In [ ]:
# JAVIEL ALEXANDER HIDALGO MATRICULA 21-EISN-2-019
# AQUI REALICE LO SIGUIENTE: En esta parte instalé todas las librerías necesarias para poder entrenar y ejecutar el modelo, TensorFlow, los datasets, ngrok, Flask y Pillow listos en el entorno.

!pip install tensorflow tensorflow-datasets pyngrok flask pillow


In [ ]:
# JAVIEL ALEXANDER HIDALGO MATRICULA 21-EISN-2-019
# AQUI REALICE LO SIGUIENTE: En esta parte cargué el dataset MNIST y apliqué una función para normalizar las imágenes y dejarlas listas para el entrenamiento del modelo.

import tensorflow as tf
import tensorflow_datasets as tfds

def preprocess_mnist(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

mnist_train, mnist_test = tfds.load(
    'mnist', split=['train', 'test'], as_supervised=True
)

mnist_train = mnist_train.map(preprocess_mnist)
mnist_test = mnist_test.map(preprocess_mnist)

print("MNIST listo")


In [ ]:
# JAVIEL ALEXANDER HIDALGO MATRICULA 21-EISN-2-019
# AQUI REALICE LO SIGUIENTE: cargué el dataset QuickDraw desde la nube y lo preparé para usarlo.

import numpy as np
import requests
import io

url = "https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/cat.npy"
r = requests.get(url)
X_qd = np.load(io.BytesIO(r.content))

X_qd = X_qd / 255.0
X_qd = X_qd.reshape(-1, 28, 28)

print("QuickDraw cargado:", len(X_qd))


In [ ]:
# JAVIEL ALEXANDER HIDALDO MATRICULA 21-EISN-2-019
# AQUI REALICE LO SIGUIENTE: Unifiqué los datos de MNIST y QuickDraw, generé sus etiquetas y preparé ambos conjuntos para el entrenamiento del modelo.

y_mnist = np.zeros(len(mnist_train), dtype=int)
y_qd = np.ones(len(X_qd), dtype=int)

X_mnist = np.array([x[0].numpy().reshape(28, 28) for x in mnist_train])

X_train = np.concatenate([X_mnist, X_qd])
y_train = np.concatenate([y_mnist, y_qd])

print("X_train:", X_train.shape)
print("y_train:", y_train.shape)


In [ ]:
# JAVIEL ALEXANDER HIDALGO MATRICULA 21-EISN-2-019
# AQUI REALICE LO SIGUIENTE: Construí el modelo CNN usando Keras, definiendo sus capas, la configuración y mostrando su resumen.

from tensorflow.keras import layers, models

NUM_CLASSES = 2

model = models.Sequential([
    layers.Input(shape=(28, 28)),
    layers.Reshape((28, 28, 1)),
    layers.Conv2D(32, 3, activation="relu"),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation="relu"),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(64, activation="relu"),
    layers.Dense(NUM_CLASSES, activation="softmax")
])

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

In [ ]:
# JAVIEL ALEXANDER HIDALGO MATRICULA 21-EISN-2-019
# AQUI REALICE LO SIGUIENTE: Dividí los datos en entrenamiento y validación, preparé los datasets y entrené el modelo guardando el resultado final.

from sklearn.model_selection import train_test_split

X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
    X_train, y_train, test_size=0.2, shuffle=True
)

train_ds = tf.data.Dataset.from_tensor_slices((X_train_split, y_train_split))\
    .shuffle(1000)\
    .batch(64)

val_ds = tf.data.Dataset.from_tensor_slices((X_val_split, y_val_split))\
    .batch(64)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=3
)

model.save("modelo_dibujos.h5")
print("Modelo guardado")
